In [1]:
from os import name
import socket
from ahrs.filters import Madgwick
import pandas as pd
import numpy as np

# Define the UDP server's IP address and port to bind to
server_ip = "0.0.0.0"  # Listen on all available network interfaces
server_port = 1234  # Replace with the port you want to use

# Create a UDP socket
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind the socket to the server IP and port
udp_socket.bind((server_ip, server_port))

print(f"UDP server is listening on {server_ip}:{server_port}")
madgwick = Madgwick()
Q = np.array([[1., 0., 0., 0.]])

def read_data():
    data, client_address = udp_socket.recvfrom(4096)  # Adjust the buffer size as needed
    gyro = []
    accel = []
    data = data.decode('utf-8').splitlines()
    for i in data:
        i = i.split(',')
        accel.append([float(i[0]), float(i[1]), float(i[2])])
        gyro.append([float(i[3]), float(i[4]), float(i[5])])
    gyro = pd.DataFrame(gyro, columns=['x', 'y', 'z'])
    accel = pd.DataFrame(accel, columns=['x', 'y', 'z'])
    return pd.concat([gyro, accel], keys=['gyro', 'accel'], axis=1)

def calibrate(n_samples=200):
    df = pd.DataFrame()
    for i in range(n_samples):
        df = pd.concat([df, read_data()], ignore_index=True)
        if i % (n_samples/10) == 0:
            print(f"Calibrating... {i}/{n_samples}")
    df = df.mean()
    df['accel'] = np.array([0, 0, 0])
    return df
    


UDP server is listening on 0.0.0.0:1234


In [2]:
calib = calibrate()
data = read_data()

Calibrating... 0/200
Calibrating... 20/200
Calibrating... 40/200
Calibrating... 60/200
Calibrating... 80/200
Calibrating... 100/200
Calibrating... 120/200
Calibrating... 140/200
Calibrating... 160/200
Calibrating... 180/200


In [5]:
df = data - calib
df

gyro                         accel                    
           x         y         z         x         y         z
0   0.003996 -0.001203 -0.000213  0.818829  0.785310 -9.739761
1   0.003996  0.000928  0.001918  0.809253  0.780522 -9.797223
2  -0.000265  0.001993  0.000852  0.804464  0.770945 -9.802011
3  -0.001330  0.001993 -0.000213  0.799676  0.770945 -9.806800
4  -0.000265 -0.000137  0.000852  0.780522  0.790099 -9.792435
5  -0.001330 -0.000137  0.002983  0.785310  0.804464 -9.778069
6  -0.003461  0.000928  0.002983  0.818829  0.837983 -9.811588
7  -0.003461  0.000928  0.002983  0.818829  0.837983 -9.811588
8   0.000800 -0.000137 -0.000213  0.790099  0.790099 -9.773280
9   0.002931  0.000928 -0.001278  0.804464  0.794887 -9.782857
10  0.001866  0.003059 -0.001278  0.780522  0.814041 -9.778069
11 -0.000265  0.004124 -0.001278  0.766156  0.804464 -9.758915
12 -0.002396  0.003059 -0.001278  0.790099  0.794887 -9.811588
13 -0.002396 -0.001203 -0.000213  0.751791  0.799676 -9.835530
14 -0.002396 -0.001203 -0.000213  0.751791  0.799676 -9.835530
15  0.001866 -0.001203  0.000852  0.790099  0.814041 -9.754127
16  0.001866 -0.001203  0.000852  0.790099  0.814041 -9.754127
17 -0.003461  0.003059  0.001918  0.828406  0.770945 -9.754127
18 -0.003461  0.000928  0.001918  0.804464  0.780522 -9.802011
19 -0.000265 -0.001203  0.000852  0.785310  0.794887 -9.802011

In [233]:
pd.Series(np.zeros(shape=(6)),index=tuple(calib.index))
tuple(calib.index)

(('gyro', 'x'),
 ('gyro', 'y'),
 ('gyro', 'z'),
 ('accel', 'x'),
 ('accel', 'y'),
 ('accel', 'z'))

In [198]:
for k,b in df.iterrows():
    print(b)
    break

gyro   x   -0.002361
       y   -0.001023
       z    0.000761
accel  x    0.914599
       y    1.192331
       z   -9.768492
Name: 0, dtype: float64
0
